## Import libraries

In [ ]:
import os
os.chdir('/workspaces/automl/src')

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from sklearn.datasets import fetch_california_housing


## Preprocessing

In [ ]:
from automl.feature_selection.pipe import PreprocessingPipeline, ValTestsPipeline
from automl.utils.utils import split_data

In [ ]:
target_col = 'MedHouseVal'
test_size = 0.2
random_state = 42
val_split_col = 'is_test_for_val'

In [ ]:
california_data = fetch_california_housing(as_frame=True)
df = pd.concat([california_data.data, pd.DataFrame(california_data.target)], axis=1)

In [ ]:
df.head()

In [ ]:
train, test = split_data(df, target_col, test_size=test_size, random_state=random_state)
X_train, y_train = train.drop(target_col, axis=1), train[target_col]
X_test, y_test = test.drop(target_col, axis=1), test[target_col]

In [ ]:
preprocessing_pipe = PreprocessingPipeline()

In [ ]:
preprocessing_pipe.fit(X_train, y_train)
X_train_prep = preprocessing_pipe.transform(X_train)
X_train_prep.columns = [col.lstrip('remainder__') for col in X_train_prep.columns]
X_test_prep = preprocessing_pipe.transform(X_test)
X_test_prep.columns = [col.lstrip('remainder__') for col in X_test_prep.columns]
print(X_train_prep.shape, X_test_prep.shape)

In [ ]:
val_tests_pipe = ValTestsPipeline()
X_train_prep[val_split_col] = 0
X_test_prep[val_split_col] = 1
X_for_val_tests = pd.concat([X_train_prep, X_test_prep], axis=0)

In [ ]:
val_tests_pipe.fit(X_for_val_tests)

X_train_after_val = val_tests_pipe.transform(X_train_prep).drop(val_split_col, axis=1)
X_test_after_val = val_tests_pipe.transform(X_test_prep).drop(val_split_col, axis=1)
print(X_train_after_val.shape, X_test_after_val.shape)

In [ ]:
train_before_fs = X_train_after_val.join(y_train)
test_before_fs = X_test_after_val.join(y_test)

## Feature Selection

In [ ]:
from automl.feature_selection.transformers import FeatureSelectionTransformer

In [ ]:
fst_rfa = FeatureSelectionTransformer(task_type='reg', target_colname=target_col, 
                                      metric_name='mae', metric_direction='minimize', timeout=120, random_state=42,
                                      strategy='RFA')

In [ ]:
fst_rfa.fit(train_before_fs, test_before_fs)

In [ ]:
fst_rfe = FeatureSelectionTransformer(task_type='reg', target_colname=target_col, 
                                      metric_name='mae', metric_direction='minimize', timeout=120, random_state=42,
                                      model='lama', strategy='RFE')

In [ ]:
fst_rfe.fit(train_before_fs, test_before_fs)

In [ ]:
fst_pfi = FeatureSelectionTransformer(task_type='reg', target_colname=target_col, 
                                      metric_name='mae', metric_direction='minimize', timeout=120, random_state=42,
                                      model='lama', strategy='PFI')

In [ ]:
fst_pfi.fit(train_before_fs, test_before_fs)

In [ ]:
train_final = fst_rfa.transform(train_before_fs)
test_final = fst_rfa.transform(test_before_fs)

In [ ]:
model = fst_rfa.train_lama_model(train_final, test_final)

In [ ]:
metric_train, metric_test = fst_rfa.calc_metric(train_final, test_final)

In [ ]:
print(f'Метрика train: {metric_train}\n Метрика test: {metric_test}')